In [ ]:
import os
from pathlib import Path

BASE_DIR = Path(os.path.abspath("")).resolve()
os.chdir(BASE_DIR)

print(f"Working directory set to: {BASE_DIR}")
%cd ..BASE_DIR


In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

In [ ]:
%cd ..
!git clone https://github.com/academy21/TC-Satellite-DataSet.git
%cd TC-Satellite-DataSet

In [ ]:
from pathlib import Path
base = BASE_DIR / 'TC-Satellite-DataSet'
print(base)

print("📂 TXT Folder 1:", list((base / 'TC_by_Classes_txt_1').glob('*')))
print("📂 TXT Folder 2:", list((base / 'TC_by_Classes_txt_2').glob('*')))
print("📂 Labels Folder:", list((base / 'labels').glob('*')))


In [ ]:
sample_path = base / 'TC_by_Classes_txt_1' / 'mt201910191400_ir1.txt'
with open(sample_path) as f:
    print(f.read())

In [ ]:
from pathlib import Path

# ✅ Define the base directory
base = BASE_DIR / 'TC-Satellite-DataSet'
# ✅ Define the folder paths
txt_folders = [
    base / 'TC_by_Classes_txt_1',
    base / 'TC_by_Classes_txt_2'
]

# ✅ Define where your images are located
images_dir = base / 'images'

# ✅ Define image matching function
def find_image_file(stem):
    for ext in ['.jpg', '.png', '.jpeg']:
        candidate = images_dir / f"{stem}_pro{ext}"
        if candidate.exists():
            return candidate
    return None

# ✅ Now find missing images
missing = []
for txt_folder in txt_folders:
    for label_file in txt_folder.glob('*.txt'):
        img_file = find_image_file(label_file.stem)
        if img_file is None:
            missing.append(label_file.name)

# ✅ Output
print("❌ Total missing:", len(missing))
print("🔍 Example missing:", missing[:5])


In [ ]:
from pathlib import Path

base = BASE_DIR / 'TC-Satellite-DataSet'
image_folders = [
    base / 'TC_by_Classes_jpg_1',
    base / 'TC_by_Classes_jpg_2', 
    base / 'TC_by_Classes_jpg_3',
    base / 'TC_by_Classes_jpg_4',
    base / 'TC_by_Classes_jpg_5',
]

image_files = []
for folder in image_folders:
    if folder.exists():
        image_files.extend(folder.glob('*.jpg'))

print("✅ Total images found:", len(image_files))
print("🖼️ Example image filenames:", [img.name for img in image_files[:5]])


In [ ]:
def find_image_file(base_name):
    for folder in image_folders:
        # Check for image with "_pro.jpg" suffix
        image_path = folder / (base_name + '_pro.jpg')
        if image_path.exists():
            return image_path
        # Fallback: check plain .jpg
        image_path = folder / (base_name + '.jpg')
        if image_path.exists():
            return image_path
    return None


In [ ]:
import os
from pathlib import Path
from shutil import copy2
from PIL import Image

base = BASE_DIR / 'TC-Satellite-DataSet'
txt_folders = [base / f'TC_by_Classes_txt_{i}' for i in range(1, 6)]
image_folders = [base / f'TC_by_Classes_jpg_{i}' for i in range(1, 6)]

output_img_dir = BASE_DIR / 'dataset'/ 'images' / 'train'
output_lbl_dir = BASE_DIR / 'dataset'/ 'labels' / 'train'
output_img_dir.mkdir(parents=True, exist_ok=True)
output_lbl_dir.mkdir(parents=True, exist_ok=True)

def find_image_file(base_name):
    for folder in image_folders:
        pro_path = folder / f'{base_name}_pro.jpg'
        if pro_path.exists():
            return pro_path
        jpg_path = folder / f'{base_name}.jpg'
        if jpg_path.exists():
            return jpg_path
    return None

skipped = 0
written = 0

for txt_folder in txt_folders:
    for label_path in txt_folder.glob('*.txt'):
        base_name = label_path.stem
        image_path = find_image_file(base_name)

        if image_path is None:
            skipped += 1
            continue

        try:
            with Image.open(image_path) as im:
                width, height = im.size
        except:
            skipped += 1
            continue

        with open(label_path) as f:
            lines = f.readlines()

        label_lines = []
        for line in lines:
            parts = line.strip().split(',')
            if len(parts) != 5:
                continue
            cls = int(parts[0].replace('Class_', '').strip())
            x, y, w, h = map(float, parts[1:])
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            norm_w = w / width
            norm_h = h / height
            label_lines.append(f"{cls} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}")

        # Save image and label
        new_img_path = output_img_dir / image_path.name
        new_lbl_path = output_lbl_dir / (image_path.stem + '.txt')
        copy2(image_path, new_img_path)
        with open(new_lbl_path, 'w') as f_out:
            f_out.write('\n'.join(label_lines))

        written += 1

print(f"✅ Conversion done. Images/Labels written: {written}")
print(f"❌ Skipped (missing or invalid): {skipped}")


In [ ]:
print("🖼️ Total images:", len(list(output_img_dir.glob('*.jpg'))))
print("🏷️ Total labels:", len(list(output_lbl_dir.glob('*.txt'))))

In [ ]:
#❌ use it when it mostly needed not for casual running

import shutil

dataset_folder = BASE_DIR / 'dataset'

try:
    shutil.rmtree(dataset_folder)
    print(f"Folder '{dataset_folder}' deleted successfully.")
except FileNotFoundError:
    print(f"Folder '{dataset_folder}' not found.")
except OSError as e:
    print(f"Error deleting folder '{dataset_folder}': {e}")


In [ ]:
import random, shutil

val_split = 0.1  # 10% validation
train_imgs = list(output_img_dir.glob('*.jpg'))
random.shuffle(train_imgs)
val_count = int(len(train_imgs) * val_split)

val_img_dir = BASE_DIR / 'dataset'/ 'images' / 'val'
val_lbl_dir = BASE_DIR / 'dataset'/ 'labels' / 'val'

val_img_dir.mkdir(parents=True, exist_ok=True)
val_lbl_dir.mkdir(parents=True, exist_ok=True)

for img_path in train_imgs[:val_count]:
    lbl_path = output_lbl_dir / (img_path.stem + '.txt')
    shutil.move(str(img_path), val_img_dir / img_path.name)
    shutil.move(str(lbl_path), val_lbl_dir / lbl_path.name)

print("✅ Split complete.")



In [ ]:
import re
from pathlib import Path

txt_folders = [BASE_DIR / 'TC-Satellite-DataSet' / f'TC_by_Classes_txt_{i}' for i in range(1, 6)]
class_ids = set()

for folder in txt_folders:
    for file in folder.glob('*.txt'):
        with open(file) as f:
            for line in f:
                match = re.match(r'Class_(\d+)', line.strip())
                if match:
                    class_ids.add(int(match.group(1)))

sorted_classes = sorted(class_ids)
print(f"✅ Total classes found: {len(sorted_classes)}")
print("📌 Classes:", [f"Class_{i}" for i in sorted_classes])


In [ ]:
from pathlib import Path

label_dir = BASE_DIR / 'dataset' / 'labels' / 'train'
class_ids = set()

for label_file in label_dir.glob('*.txt'):
    with open(label_file) as f:
        for line in f:
            if line.strip():
                class_id = int(line.strip().split()[0])
                class_ids.add(class_id)

max_class_id = max(class_ids)
sorted_classes = sorted(class_ids)

print(f"✅ Highest class ID found: {max_class_id}")
print(f"🔢 Set nc to: {max_class_id + 1}")
print("📌 Classes:", [f"Class_{i}" for i in sorted_classes])

In [ ]:

import yaml

data = {
    'path': '/content/dataset',  # Replace with the actual path to your dataset
    'train': 'images/train',
    'val': 'images/val',
    'nc': 8,  # Number of classes (replace with the actual number)
    'names': ['Class_0', 'Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7']  # Class names (replace with the actual class names)
}

yolov5_dir = BASE_DIR / 'yolov5'
yaml_file_path = yolov5_dir / 'cyclone.yaml'

yaml_file_path.parent.mkdir(parents=True, exist_ok=True)

with open(yaml_file_path, 'w') as f:
    yaml.dump(data, f, indent=2)

print(f"Successfully created cyclone.yaml at: {yaml_file_path}")


In [ ]:
%cd ../yolov5
!python train.py --img 640 --batch 8 --epochs 100 --data cyclone.yaml --weights yolov5l.pt --name cyclone_detector --cache

In [ ]:
!python val.py --weights runs/train/cyclone_detector/weights/best.pt --data cyclone.yaml --img 640

In [ ]:
!python detect.py --weights runs/train/cyclone_detector/weights/best.pt --img 640 --source ../dataset/images/val --conf 0.25

In [ ]:
import pandas as pd

df = pd.read_csv('runs/train/cyclone_detector/results.csv')
print(df.columns.tolist())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('runs/train/cyclone_detector/results.csv')

df.columns = df.columns.str.strip()

print(df.head())

df['epoch'] = pd.to_numeric(df['epoch'], errors='coerce')

df = df.dropna(subset=['epoch'])

print(df[['epoch', 'metrics/precision', 'metrics/recall', 'metrics/mAP_0.5']].head())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('runs/train/cyclone_detector/results.csv')
df.columns = df.columns.str.strip()

plt.figure(figsize=(10, 6))
plt.scatter(df['epoch'], df['metrics/mAP_0.5'], label='mAP@0.5', marker='o')
plt.scatter(df['epoch'], df['metrics/precision'], label='Precision', marker='s')
plt.scatter(df['epoch'], df['metrics/recall'], label='Recall', marker='^')

plt.title('Training Metrics')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
!python export.py --weights runs/train/cyclone_detector/weights/best.pt --include onnx

In [ ]:
!python export.py --weights runs/train/cyclone_detector2/weights/best.pt --include tflite

In [ ]:
from google.colab import files
files.download('runs/train/cyclone_detector2/weights/best.onnx')  # or .tflite

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>